In [1]:
import torch as th
import sys
import os
import certifi
import matplotlib.pyplot as plt
from lpe.methods import MHIS
from lpe.method_utils import *
from lpe.utils import Transformer
from lpe.utils import datasets as lpe_datasets
from lpe.utils.model_adapter import ModelAdapter
from lpe.da_mhis import DA_MHIS
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# Set device
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# Set model name
model_name = 'openai-community/gpt2' # or 'gelu-1l' or whatever

In [ ]:
# Run if toy model
model = Transformer.from_pretrained(model_name).to(device)

In [4]:
# Run if HF model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
hf_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model = ModelAdapter(hf_model, tokenizer, device=device)

In [5]:
dist_name = 'camel'
behavior = distribution_registry[dist_name](model.tokenizer, device=model.device)
orig_dists = behavior.input_dists(n_reps=N_REPS_DICT[dist_name])

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/53 [00:00<?, ?it/s]

100%|██████████| 1024/1024 [00:06<00:00, 160.37it/s]


In [6]:
target = 123  # some token id
temp = 1.0    # tune as needed
ests = DA_MHIS(model, orig_dists, target, temp=temp, n_samples=2**12, burn_in=2**9, show_progress=True)
print(ests)

100%|██████████| 640/640 [00:32<00:00, 19.51it/s]

0.0
